# 80. ID番号への変換
問題51で構築した学習データ中の単語にユニークなID番号を付与したい．学習データ中で最も頻出する単語に1，2番目に頻出する単語に2，……といった方法で，学習データ中で2回以上出現する単語にID番号を付与せよ．そして，与えられた単語列に対して，ID番号の列を返す関数を実装せよ．ただし，出現頻度が2回未満の単語のID番号はすべて0とせよ．

In [174]:
import pandas as pd
import re
from collections import Counter
from tqdm import tqdm
tqdm.pandas()


def tokenize(doc):
    tokens = doc.split(' ')
    return tokens


def normalize(doc):
    doc = re.sub(r"[',.]", '', doc)   # 記号を削除
    doc = re.sub(r" {2,}", ' ', doc)  # 2回以上続くスペースを削除
    doc = re.sub(r" *?$", '', doc)    # 行頭と行末のスペースを削除
    doc = re.sub(r"^ *?", '', doc)
    doc = doc.lower()                 # 小文字に統一
    return doc


def token2id(token):
    return token2id_dic[token]


columns = ('category', 'title')

train = pd.read_csv('../../data/NewsAggregatorDataset/train.txt',
                    names=columns, sep='\t')


docs = [normalize(doc) for doc in train.title.values.tolist()]
tokens = [tokenize(doc) for doc in docs]
tokens = sum(tokens, [])  # flat list
counter = Counter(tokens)

token2id_dic = {}
vocab_size = len(counter)
for index, (token, freq) in enumerate(counter.most_common(), 1):
    if freq < 2:
        token2id_dic[token] = 0
    else:
        token2id_dic[token] = index


In [175]:
token2id('the')
vocab_size

16901

# 81. RNNによる予測
ID番号で表現された単語列x=(x1,x2,…,xT)がある．ただし，Tは単語列の長さ，xt∈ℝVは単語のID番号のone-hot表記である（Vは単語の総数である）．再帰型ニューラルネットワーク（RNN: Recurrent Neural Network）を用い，単語列xからカテゴリyを予測するモデルとして，次式を実装せよ...

In [50]:
def preprocessor(doc):
    doc = normalize(doc)    
    tokens = tokenize(doc)
    return tokens


def tokens2ids(tokens):
    tokens = [token2id(token) for token in tokens]
    return torch.tensor(tokens, dtype=torch.int64)

In [177]:
import torch


train['tokens'] = train.title.apply(preprocessor)
X_train = train.tokens.apply(tokens2ids)
X_train[0]

tensor([   8,    0, 2416, 1604, 2143,    5, 1605,    4,  745])

In [259]:
import torch.nn as nn

dw = 300
dh = 50
L = 4

class RNN(nn.Module):
    def __init__(self, data_size, hidden_size, output_size, vocab_size):
        super(RNN, self).__init__()
        self.emb = torch.nn.Embedding(vocab_size, data_size)
        self.rnn = torch.nn.RNN(dw, dh, nonlinearity='relu')
        self.liner = nn.Linear(hidden_size, output_size)


    def forward(self, data, last_hidden):           # data: (max_len)
        data = self.emb(data)                       # data: (max_length, dw)
        y, hidden = self.rnn(data, last_hidden)     # y: (max_len, dh), hidden: (max_len, dh)
        y = y[:,-1,:]
        y = self.liner(y)
        y = torch.softmax(y, dim=1)
        return y, hidden


In [260]:
from torch.nn.utils.rnn import pad_sequence

max_len = train.tokens.apply(len).max()
model = RNN(dw, dh, L, vocab_size)

label2int = {'b': 0, 't': 1, 'e': 2, 'm': 3}
Y_train = train.category.map(label2int)
Y_train = torch.tensor(Y_train).int()

inputs = X_train
labels = Y_train


inputs = pad_sequence(inputs, batch_first=True)
h0 = torch.zeros(1, 121, dh, dtype=torch.float32)

outputs, hidden = model(inputs, h0)
print(outputs.size())
print(hidden.size())

torch.Size([10672, 4])
torch.Size([1, 121, 50])
